<a href="https://colab.research.google.com/github/linadevinci/td02-Lina-yahiaoui/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lien Github :
https://github.com/linadevinci/td02-Lina-yahiaoui

In [ ]:
!pip install ecdsa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
import secrets
import hashlib
import binascii
import argparse
import sys

import hashlib
import hmac
import struct
from ecdsa import SigningKey, SECP256k1
from ecdsa.util import string_to_number

In [ ]:

# Charger la liste de mots BIP39
with open('bip39_words.txt', 'r') as f:
    BIP39_WORDS = f.read().splitlines()



In [ ]:
def generate_seed(bits: int = 128):
    """Génère un entier aléatoire sécurisé pour la seed"""
    return secrets.randbits(bits)


In [ ]:

def seed_to_binary(seed: int):
    """Convertit la seed en binaire"""
    return bin(seed)[2:].zfill(128)  # 128 bits, pad with leading zeros if necessary

def split_in_chunks(binary_seed: str, chunk_size: int = 11):
    """Découpe la seed binaire en lots de 11 bits"""
    return [binary_seed[i:i+chunk_size] for i in range(0, len(binary_seed), chunk_size)]

def chunks_to_mnemonic(chunks):
    """Associe chaque chunk à un mot BIP39."""
    mnemonic = []
    for chunk in chunks:
        index = int(chunk, 2)  # Convertir le chunk binaire en entier
        mnemonic.append(BIP39_WORDS[index])  # Associer à un mot BIP39
    return ' '.join(mnemonic)



def mnemonic_to_seed(mnemonic):
    """Convertit une phrase mnémonique en seed binaire."""
    words = mnemonic.split(' ')
    binary_seed = ''
    for word in words:
        if word not in BIP39_WORDS:
            raise ValueError(f"Le mot '{word}' n'est pas dans la liste BIP39.")
        index = BIP39_WORDS.index(word)
        binary_seed += bin(index)[2:].zfill(11)  # Convertir chaque mot en binaire
    return int(binary_seed, 2)




In [ ]:
print("Bienvenue dans le générateur BIP39 Seed.")
print("1. Générer une nouvelle seed")
print("2. Importer une phrase mnémonique")
choice = input("Veuillez choisir une option (1 ou 2): ")

if choice == '1':
    # Générer une seed et afficher la phrase mnémonique
    seed = generate_seed()
    binary_seed = seed_to_binary(seed)
    chunks = split_in_chunks(binary_seed)
    mnemonic = chunks_to_mnemonic(chunks)
    print(f"\nSeed générée (entier): {seed}")
    print(f"Phrase mnémonique générée: {mnemonic}")
elif choice == '2':
    # Demander une phrase mnémonique à l'utilisateur et afficher la seed correspondante
    mnemonic = input("Veuillez entrer la phrase mnémonique: ")
    try:
        seed = mnemonic_to_seed(mnemonic)
        print(f"\nPhrase mnémonique importée: {mnemonic}")
        print(f"Seed correspondante (entier): {seed}")
    except ValueError as e:
        print(f"Erreur: {e}")
else:
    print("Option invalide. Veuillez relancer le programme.")

Bienvenue dans le générateur BIP39 Seed.
1. Générer une nouvelle seed
2. Importer une phrase mnémonique
Veuillez choisir une option (1 ou 2): 2
Veuillez entrer la phrase mnémonique: add oppose tuna garlic kid poverty erupt carbon wage protect hamster artwork

Phrase mnémonique importée: add oppose tuna garlic kid poverty erupt carbon wage protect hamster artwork
Seed correspondante (entier): 68077392771901247489479967149905291369


In [ ]:

# Constants
BIP32_HARDENED_OFFSET = 0x80000000
BIP32_SEED_KEY = b'Bitcoin seed'

# Utilitaire pour générer le hachage SHA512
def hmac_sha512(key, data):
    return hmac.new(key, data, hashlib.sha512).digest()

# Générer la master private key et le chain code à partir de la seed
def generate_master_key(seed):
    hashed = hmac_sha512(BIP32_SEED_KEY, seed)
    master_private_key = hashed[:32]
    chain_code = hashed[32:]
    return master_private_key, chain_code

# Générer la master public key à partir de la private key
def private_to_public(private_key):
    # Utilise ecdsa pour générer une clé publique
    sk = SigningKey.from_string(private_key, curve=SECP256k1)
    vk = sk.verifying_key
    return b'\x04' + vk.to_string()  # Préfixe avec 0x04 pour indiquer une clé publique non compressée

# Générer une clé enfant à partir d'une clé privée et d'un index (normal ou dérivé)
def derive_child_key(parent_private_key, parent_chain_code, index):
    if index >= BIP32_HARDENED_OFFSET:
        # Clé dérivée (hardened): HMAC avec la clé privée
        data = b'\x00' + parent_private_key + struct.pack('>L', index)
    else:
        # Clé non dérivée: HMAC avec la clé publique
        parent_public_key = private_to_public(parent_private_key)
        data = parent_public_key + struct.pack('>L', index)

    hashed = hmac_sha512(parent_chain_code, data)
    child_private_key = (string_to_number(parent_private_key) + string_to_number(hashed[:32])) % SECP256k1.order
    child_private_key = child_private_key.to_bytes(32, 'big')
    child_chain_code = hashed[32:]
    return child_private_key, child_chain_code

# Générer une clé enfant à un index N
def derive_child_at_index(master_private_key, master_chain_code, index):
    return derive_child_key(master_private_key, master_chain_code, index)

# Générer une clé enfant à l'index N et au niveau de dérivation M
def derive_child_at_path(master_private_key, master_chain_code, path):
    private_key = master_private_key
    chain_code = master_chain_code
    for index in path:
        private_key, chain_code = derive_child_key(private_key, chain_code, index)
    return private_key, chain_code

# Exemple d'utilisation
seed = b'some random seed that is at least 128 bits long'
master_private_key, master_chain_code = generate_master_key(seed)

print("Master Private Key:", master_private_key.hex())
print("Master Chain Code:", master_chain_code.hex())

master_public_key = private_to_public(master_private_key)
print("Master Public Key:", master_public_key.hex())

# Générer une clé enfant (index 0)
child_private_key, child_chain_code = derive_child_at_index(master_private_key, master_chain_code, 0)
print("\nChild Private Key (index 0):", child_private_key.hex())
print("Child Chain Code (index 0):", child_chain_code.hex())

# Générer une clé enfant dérivée à l'index N=1
child_private_key_n1, child_chain_code_n1 = derive_child_at_index(master_private_key, master_chain_code, 1)
print("\nChild Private Key (index 1):", child_private_key_n1.hex())
print("Child Chain Code (index 1):", child_chain_code_n1.hex())

# Générer une clé enfant au niveau de dérivation M=0 avec l'index N=1 (path = [0, 1])
child_private_key_m0_n1, child_chain_code_m0_n1 = derive_child_at_path(master_private_key, master_chain_code, [0, 1])
print("\nChild Private Key (path [0, 1]):", child_private_key_m0_n1.hex())
print("Child Chain Code (path [0, 1]):", child_chain_code_m0_n1.hex())


Master Private Key: b14d27e69cabfeb40ee4571625ce99fb9e45d9753531ba51ddfab039464d076a
Master Chain Code: f9518863da1c439bcda8081c05de31dc6624918dac8ced79cb593e3ace08c833
Master Public Key: 044792c3954f60511ecdfa7bf44f9552159be6ab1f5cfc3e44e9072f069b96f6917a3cebefeea31049655c30b92e829df31eeaa4dca4bf2c56c15fbc4af7eb3120

Child Private Key (index 0): 153eeabf1e158d9f32d80d998b51eb44d48aca1f1e80149c9f17d8fddd8c524c
Child Chain Code (index 0): fa27f03db00fa850a028e726db00a343e178a5397b661e0b22d9465286189b28

Child Private Key (index 1): fbc8a029f8ea889306b7c3053b7e361c317bd862339d1f745ce32f2018d3816d
Child Chain Code (index 1): 10c174300664866a0258f9cc30797f133121288e38d7cf8ffa2d0c66c9f8e8d1

Child Private Key (path [0, 1]): 067e646d6cbb9a19d39d542d031f550895d794c33bf24ade757c5339fb25f9e5
Child Chain Code (path [0, 1]): 6a16121c334d2f44de289fb46fc6a904be7ec4d183ca561aa2924bc82389056e
